In [5]:
# @title Setup
from google.colab import auth
# from google.cloud import bigquery
from google.colab import data_table
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from datetime import datetime, timedelta

# Verfications

In [8]:
verifications_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/verifications/verifications.parquet')

In [9]:
verifications_df = verifications_df.sort_values(by='fid')

In [10]:
verified_df = verifications_df[verifications_df.deleted_at.isnull()].drop_duplicates(['fid'])[['fid', 'address', 'timestamp']]

In [11]:
verified_df.head()

,fid,address,timestamp
29197,1,0x86924c37a93734e8611eb081238928a9d18a63c0,2023-07-22 00:08:54+00:00
30132,2,0x91031dcfdea024b4d51e775486111d2b2a715871,2023-04-28 17:42:20+00:00
31115,3,0x8fc5d6afe572fefc4ec153587b63ce543f6fa2ea,2023-09-27 23:28:30+00:00
33023,5,b5ewlms1kuljzi6slzjzeftj6wu5wqclu6kfgdrvgaxd,2024-02-23 04:31:20+00:00
35793,8,0x17cd072cbd45031efc21da538c783e0ed3b25dcc,2023-06-12 23:44:04+00:00


# Links Data


In [12]:
links_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/links/links.parquet')

In [13]:
links_df = links_df[links_df.type.isin(['follow', 'unfollow'])]
links_df = links_df.drop_duplicates(['target_fid', 'fid'], keep=False)

In [14]:
links_df.head()

,fid,timestamp,target_fid,type,deleted_at
0,19099,2024-04-20 22:06:08+00:00,461009,follow,NaT
1,252634,2024-05-03 10:06:31+00:00,250833,follow,NaT
2,418937,2024-04-06 09:26:16+00:00,415186,follow,NaT
3,385938,2024-04-17 19:43:30+00:00,403156,follow,NaT
4,500406,2024-04-29 11:21:25+00:00,484567,follow,NaT


# Profile Data

In [15]:
profiles_df = pd.read_parquet(f'/content/drive/My Drive/farcaster_bigquery/profiles/profiles.parquet')

In [16]:
profiles_df.head()

,fid,last_updated_at,data,custody_address,date
0,504692,2024-04-28 08:29:33+00:00,None,0x77b0352100ebdb86a45557296e05721e48cc6d01,2024-04-28
1,530839,2024-05-11 10:39:33+00:00,None,0xef7c5ee8eb10cd4ae60611506532e1a6a279c070,2024-05-11
2,521202,2024-05-08 11:05:41+00:00,None,0x74d8967e812de34702ecd3d453a44bf37440b10b,2024-05-08
3,530583,2024-05-11 08:40:01+00:00,None,0xfefa4f4aebb13ebe75ac5ffb7374ea6c2478e350,2024-05-11
4,531932,2024-05-12 02:14:13+00:00,None,0x3fe713ae9fbcb4f86f960716e6316d418d15e6a2,2024-05-12


In [17]:
profiles_df.size

2696695

In [18]:
profiles_df

,fid,last_updated_at,data,custody_address,date
0,504692,2024-04-28 08:29:33+00:00,None,0x77b0352100ebdb86a45557296e05721e48cc6d01,2024-04-28
1,530839,2024-05-11 10:39:33+00:00,None,0xef7c5ee8eb10cd4ae60611506532e1a6a279c070,2024-05-11
2,521202,2024-05-08 11:05:41+00:00,None,0x74d8967e812de34702ecd3d453a44bf37440b10b,2024-05-08
3,530583,2024-05-11 08:40:01+00:00,None,0xfefa4f4aebb13ebe75ac5ffb7374ea6c2478e350,2024-05-11
4,531932,2024-05-12 02:14:13+00:00,None,0x3fe713ae9fbcb4f86f960716e6316d418d15e6a2,2024-05-12
...,...,...,...,...,...
539334,3348,2024-02-27 21:18:17+00:00,"{""bio"":""🫶 /heartspotting"",""display"":""empowa"",""...",None,2024-02-27
539335,292146,2024-04-09 18:46:05+00:00,"{""bio"":""🫶 Crypto , Forex 🫶"",""display"":""Amir Za...",None,2024-04-09
539336,308217,2024-05-11 16:23:15+00:00,"{""bio"":""🫶 web3 communities | 👩‍💻 @cryptopols |...",None,2024-05-11
539337,311855,2024-02-21 16:51:26+00:00,"{""bio"":""🫶🏻"",""display"":""driftwoods 🎩"",""pfp"":""ht...",None,2024-02-21


In [19]:
# Function to parse JSON strings
def parse_json_column(json_str):
    return pd.Series(json.loads(json_str))

profiles_df.data.fillna('{}', inplace=True)

# Apply the function to the 'Profile' column
expanded_columns = profiles_df['data'].apply(parse_json_column)

# Concatenate the original DataFrame with the new columns
scores_df = pd.concat([profiles_df, expanded_columns], axis=1)



In [21]:
import pandas as pd
import numpy as np

# Sample DataFrame
# data = {
#     'ID': [1, 2, 3, 4, 5],
#     'timestamp': ['2024-06-10 10:00:00', '2024-06-11 12:00:00', '2024-06-09 09:00:00', '2024-06-10 08:00:00', '2024-06-12 14:00:00']
# }

# Create the DataFrame and convert the timestamp to datetime
# df = pd.DataFrame(data)
# df['timestamp'] = pd.to_datetime(df['timestamp'])

# Function to estimate original timestamps
def estimate_original_timestamps(df):
    # Sort the DataFrame by ID and timestamp
    df_sorted = df.sort_values(by=['fid']).reset_index(drop=True)

    # Create a new column to store the estimated timestamps
    df_sorted['estimated_timestamp'] = df_sorted['last_updated_at']

    # Iterate over the DataFrame
    for i in range(len(df_sorted) - 1):
        current_id = df_sorted.at[i, 'fid']
        current_timestamp = df_sorted.at[i, 'last_updated_at']

        # Find the minimum subsequent timestamp
        subsequent_timestamps = df_sorted.loc[i+1:, 'last_updated_at']
        if not subsequent_timestamps.empty:
            min_subsequent_timestamp = subsequent_timestamps.min()

            # If the current timestamp is newer than the minimum subsequent timestamp, replace it
            if current_timestamp > min_subsequent_timestamp:
                df_sorted.at[i, 'estimated_timestamp'] = min_subsequent_timestamp

    return df_sorted

# Apply the function
estimated_df = estimate_original_timestamps(scores_df)

# Display the result
# print(estimated_df)
# scores_df.head()
# The resulting DataFrame will contain the original and estimated timestamps


,fid,last_updated_at,data,custody_address,date,username,display,pfp,bio,url
0,504692,2024-04-28 08:29:33+00:00,{},0x77b0352100ebdb86a45557296e05721e48cc6d01,2024-04-28,NaN,NaN,NaN,NaN,NaN
1,530839,2024-05-11 10:39:33+00:00,{},0xef7c5ee8eb10cd4ae60611506532e1a6a279c070,2024-05-11,NaN,NaN,NaN,NaN,NaN
2,521202,2024-05-08 11:05:41+00:00,{},0x74d8967e812de34702ecd3d453a44bf37440b10b,2024-05-08,NaN,NaN,NaN,NaN,NaN
3,530583,2024-05-11 08:40:01+00:00,{},0xfefa4f4aebb13ebe75ac5ffb7374ea6c2478e350,2024-05-11,NaN,NaN,NaN,NaN,NaN
4,531932,2024-05-12 02:14:13+00:00,{},0x3fe713ae9fbcb4f86f960716e6316d418d15e6a2,2024-05-12,NaN,NaN,NaN,NaN,NaN


In [24]:
estimated_df[['fid', 'last_updated_at', 'estimated_timestamp']].to_parquet(f'/content/drive/My Drive/farcaster_bigquery/estimate_original_timestamps.parquet')

In [23]:
estimated_df.head(30)

,fid,last_updated_at,data,custody_address,date,username,display,pfp,bio,url,estimated_timestamp
0,1,2023-07-22 01:20:44+00:00,"{""bio"":""A sufficiently decentralized social ne...",None,2023-07-22,farcaster,Farcaster,https://i.imgur.com/I2rEbPF.png,A sufficiently decentralized social network. f...,NaN,2023-03-15 06:40:17+00:00
1,2,2024-05-06 21:10:43+00:00,"{""bio"":""Technowatermelon. Elder Millenial. Bui...",None,2024-05-06,v,Varun Srinivasan,https://i.seadn.io/gae/sYAr036bd0bRpj7OX6B-F-M...,Technowatermelon. Elder Millenial. Building Fa...,NaN,2023-03-15 06:40:17+00:00
2,3,2023-08-31 16:52:50+00:00,"{""bio"":""Working on Farcaster and Warpcast."",""d...",None,2023-08-31,dwr.eth,Dan Romero,https://res.cloudinary.com/merkle-manufactory/...,Working on Farcaster and Warpcast.,NaN,2023-03-15 06:40:17+00:00
3,4,2023-03-16 17:16:26+00:00,"{""bio"":""Welcoming you to the ￼Far Side. \n\nMa...",None,2023-03-16,noah,Noah Jessop,https://lh3.googleusercontent.com/gsOZX_CswJw_...,Welcoming you to the ￼Far Side. \n\nManaging D...,NaN,2023-03-15 06:40:17+00:00
4,5,2023-07-27 05:39:34+00:00,"{""bio"":""https://mirceap.com"",""display"":""Mircea...",None,2023-07-27,mircea.eth,Mircea Pasoi,https://i.seadn.io/gcs/files/1d12ec3563a9b6aba...,https://mirceap.com,NaN,2023-03-15 06:40:17+00:00
5,6,2023-03-15 06:40:17+00:00,"{""bio"":""I can lift 100 lbs. over my head with ...",None,2023-03-15,libovness,libovness,https://lh3.googleusercontent.com/Ikka42EZ7e7u...,I can lift 100 lbs. over my head with very lit...,NaN,2023-03-15 06:40:17+00:00
6,7,2023-03-15 06:40:17+00:00,"{""bio"":"""",""display"":""kevin"",""pfp"":""https://lh3...",None,2023-03-15,kevin,kevin,https://lh3.googleusercontent.com/h6cxmp8kjwDz...,,NaN,2023-03-15 06:40:17+00:00
7,8,2024-02-11 22:44:40+00:00,"{""bio"":""Working on /zora, zora.co.\n +++???!!!...",None,2024-02-11,jacob,jacob,https://i.imgur.com/tt8uLVd.jpg,"Working on /zora, zora.co.\n +++???!!!",NaN,2023-03-15 06:40:19+00:00
8,9,2023-03-15 06:40:19+00:00,"{""bio"":"""",""display"":""b-rad.eth"",""pfp"":""https:/...",None,2023-03-15,b-rad,b-rad.eth,https://i.seadn.io/gae/oV72v-ZhWvVOtqiBlaRSQP2...,,NaN,2023-03-15 06:40:19+00:00
9,10,2023-07-04 11:09:29+00:00,"{""bio"":"""",""display"":""Elad"",""pfp"":""https://i.im...",None,2023-07-04,elad,Elad,https://i.imgur.com/MFLcSdf.jpg,,NaN,2023-03-15 06:40:20+00:00


In [43]:
import datetime

timestamp = 80529644
# timestamp = 1642445213
value = datetime.datetime.fromtimestamp(timestamp)
print(f"{value:%Y-%m-%d %H:%M:%S}")
# timestamp = 48994466
# date_time = datetime.datetime.utcfromtimestamp(timestamp)
# formatted_date = date_time.strftime("%d/%m/%Y")
# formatted_date


1972-07-21 01:20:44


In [30]:
scores_df.sort_values(by='fid')

,fid,last_updated_at,data,custody_address,date,username,display,pfp,bio,url
397945,1,2023-07-22 01:20:44+00:00,"{""bio"":""A sufficiently decentralized social ne...",None,2023-07-22,farcaster,Farcaster,https://i.imgur.com/I2rEbPF.png,A sufficiently decentralized social network. f...,NaN
503714,2,2024-05-06 21:10:43+00:00,"{""bio"":""Technowatermelon. Elder Millenial. Bui...",None,2024-05-06,v,Varun Srinivasan,https://i.seadn.io/gae/sYAr036bd0bRpj7OX6B-F-M...,Technowatermelon. Elder Millenial. Building Fa...,NaN
513452,3,2023-08-31 16:52:50+00:00,"{""bio"":""Working on Farcaster and Warpcast."",""d...",None,2023-08-31,dwr.eth,Dan Romero,https://res.cloudinary.com/merkle-manufactory/...,Working on Farcaster and Warpcast.,NaN
512249,4,2023-03-16 17:16:26+00:00,"{""bio"":""Welcoming you to the ￼Far Side. \n\nMa...",None,2023-03-16,noah,Noah Jessop,https://lh3.googleusercontent.com/gsOZX_CswJw_...,Welcoming you to the ￼Far Side. \n\nManaging D...,NaN
524583,5,2023-07-27 05:39:34+00:00,"{""bio"":""https://mirceap.com"",""display"":""Mircea...",None,2023-07-27,mircea.eth,Mircea Pasoi,https://i.seadn.io/gcs/files/1d12ec3563a9b6aba...,https://mirceap.com,NaN
...,...,...,...,...,...,...,...,...,...,...
70397,539335,2024-05-16 04:07:43+00:00,"{""bio"":""Crypto and Meme Coin Enthusiast"",""disp...",0x86c268e3b40c66275ab01c6f8bd36389e59aea25,2024-05-16,cryptoniankage,CryptonianKage,https://imagedelivery.net/BXluQx4ige9GuW0Ia56B...,Crypto and Meme Coin Enthusiast,NaN
469,539336,2024-05-16 04:08:19+00:00,{},0x1ae1e38867bc4cae316bb4937b56081461f449eb,2024-05-16,NaN,NaN,NaN,NaN,NaN
465,539337,2024-05-16 04:09:05+00:00,{},0xa2633f9816ce830b8414563a8f3e905c438d7758,2024-05-16,NaN,NaN,NaN,NaN,NaN
471,539338,2024-05-16 04:09:15+00:00,{},0x78d3e8c5c395ec1aff5431acc65a5626004549b6,2024-05-16,NaN,NaN,NaN,NaN,NaN


In [14]:
scores_df[scores_df.username.notna()].username

,fid,last_updated_at,data,custody_address,date,username,display,pfp,bio,url
1153,536445,2024-05-14 15:29:59+00:00,"{""username"":""bersserg""}",0x731e32c27ac27b804e11647b760f427e9e1058b7,2024-05-14,bersserg,NaN,NaN,NaN,NaN
1159,486232,2024-04-16 06:26:03+00:00,"{""username"":""hadinem""}",0xe1f0d058fcc5f114f97cd005a43ade1387b88b10,2024-04-16,hadinem,NaN,NaN,NaN,NaN
1160,395201,2024-03-17 17:07:42+00:00,"{""username"":""edsantos28""}",0xba503adb295c3b91b2fb26459aa7cf0800a077ce,2024-03-17,edsantos28,NaN,NaN,NaN,NaN
1168,367610,2024-02-22 11:28:07+00:00,"{""username"":""cainedith""}",0x28148cd3d8546d8478240d7b45af13ba8eef5a8e,2024-02-22,cainedith,NaN,NaN,NaN,NaN
1175,424437,2024-03-31 04:43:42+00:00,"{""username"":""yonsei""}",0x6b26f9c2121efb673937ce97e15ef708abfe57d3,2024-03-31,yonsei,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
539334,3348,2024-02-27 21:18:17+00:00,"{""bio"":""🫶 /heartspotting"",""display"":""empowa"",""...",None,2024-02-27,empowa,empowa,https://i.imgur.com/BYmE6gt.jpg,🫶 /heartspotting,NaN
539335,292146,2024-04-09 18:46:05+00:00,"{""bio"":""🫶 Crypto , Forex 🫶"",""display"":""Amir Za...",None,2024-04-09,amirzakizadeh,Amir Zakizadeh 🎩🔵🐐,https://ipfs.decentralized-content.com/ipfs/ba...,"🫶 Crypto , Forex 🫶",NaN
539336,308217,2024-05-11 16:23:15+00:00,"{""bio"":""🫶 web3 communities | 👩‍💻 @cryptopols |...",None,2024-05-11,estie,estie.eth,https://imagedelivery.net/BXluQx4ige9GuW0Ia56B...,🫶 web3 communities | 👩‍💻 @cryptopols | ☕️ coff...,NaN
539337,311855,2024-02-21 16:51:26+00:00,"{""bio"":""🫶🏻"",""display"":""driftwoods 🎩"",""pfp"":""ht...",None,2024-02-21,driftwoods,driftwoods 🎩,https://i.imgur.com/u6iRGef.jpg,🫶🏻,NaN


In [15]:
usernames = scores_df[scores_df.username.notna()].username

In [16]:
usernames

1153           bersserg
1159            hadinem
1160         edsantos28
1168          cainedith
1175             yonsei
              ...      
539334           empowa
539335    amirzakizadeh
539336            estie
539337       driftwoods
539338        alireza78
Name: username, Length: 347806, dtype: object

In [17]:
from collections import defaultdict

# Initialize a dictionary to store bigram frequencies
bigram_freq = defaultdict(int)
total_bigrams = 0

# Calculate bigram frequencies
for name in usernames:
    for i in range(len(name) - 1):
        bigram = name[i:i+2]
        bigram_freq[bigram] += 1
        total_bigrams += 1

# Convert frequencies to probabilities
bigram_likelihood = {bigram: freq / total_bigrams for bigram, freq in bigram_freq.items()}


In [23]:
import math

# Function to compute geometric mean of bigram likelihoods
def geometric_mean_likelihood(screen_name, bigram_likelihood):
    bigrams = [screen_name[i:i+2] for i in range(len(screen_name) - 1)]
    likelihoods = [bigram_likelihood.get(bigram, 1e-10) for bigram in bigrams]  # Use a small value for unseen bigrams
    if not likelihoods:
        return 0
    log_likelihoods = [math.log(l) for l in likelihoods]
    return math.exp(sum(log_likelihoods) / len(log_likelihoods))

# Example of computing geometric mean likelihood for a screen name
screen_name_example = "user123"
gm_likelihood = geometric_mean_likelihood(screen_name_example, bigram_likelihood)
print(f"Geometric Mean Likelihood for '{screen_name_example}': {gm_likelihood}")



Geometric Mean Likelihood for 'user123': 0.0024429513885392976


In [25]:
screen_name_likelihoods = {name: geometric_mean_likelihood(name, bigram_likelihood) for name in usernames}

In [26]:
screen_name_likelihoods

{'bersserg': 0.003903125947323788,
 'hadinem': 0.005674371297877218,
 'edsantos28': 0.002534334340976502,
 'cainedith': 0.005076763457580679,
 'yonsei': 0.003131092131252537,
 'hhfjkl': 0.000209666878804854,
 'sandmatters1031': 0.0026426099805131934,
 'felipeavilag': 0.0031858064918174455,
 'brucebaldillez': 0.002874383618148249,
 'gsak27': 0.0009552065121841455,
 'redmister': 0.004623020575457291,
 'skye99': 0.0008782188379574686,
 'criticalmass': 0.00441697887581767,
 'hantosfulula': 0.003259012324519389,
 'napuh': 0.0016285849255064635,
 'elpaso': 0.0030432375842575133,
 'mtao1': 0.0009945244529343856,
 'spinboy': 0.002201940803758567,
 'lela25': 0.0023259780753734135,
 'r0bert': 0.0010973508592430042,
 'denaa05': 0.0020831393542671767,
 'brittanybauer': 0.003386703005875042,
 'tendoumeisuke': 0.0032351361745197515,
 'zonderundefined': 0.004099945584280848,
 '130596': 0.000433712621579915,
 'ufebef': 0.0013254497177474281,
 'redhair': 0.003522553165072327,
 'artist216': 0.0021119234

In [29]:
test_screen_names = ["rand0m_str1ng", "legituser", "bot_account", "random123456", "warnpopulace", "etherealembrace"]

# Calculate likelihoods for test names
for name in test_screen_names:
    likelihood = geometric_mean_likelihood(name, bigram_likelihood)
    print(f"Geometric Mean Likelihood for '{name}': {likelihood}")

Geometric Mean Likelihood for 'rand0m_str1ng': 9.528386961401127e-05
Geometric Mean Likelihood for 'legituser': 0.003674517268314462
Geometric Mean Likelihood for 'bot_account': 8.785448573127886e-05
Geometric Mean Likelihood for 'random123456': 0.001661919608953817
Geometric Mean Likelihood for 'warnpopulace': 0.0023074132347715363
Geometric Mean Likelihood for 'etherealembrace': 0.004666639916016073


In [ ]:
# Drop the original 'Profile' column
scores_df.drop(columns=['data'], inplace=True)

# Convert tabular data to graph data

In [ ]:
G = nx.from_pandas_edgelist(
    links_df,
    source='fid',
    target='target_fid',
    create_using=nx.DiGraph()
)

In [ ]:
G.number_of_nodes()

410104

In [ ]:
links_df.fid.nunique()

343115

In [ ]:
links_df.target_fid.nunique()

408458

In [ ]:
G.size()

84359957

# Bot Detection

### Louvain Method

In [ ]:
import community.community_louvain

In [ ]:
!pip install python-louvain

In [ ]:
import community.community_louvain as community_louvain

In [ ]:
Gd = G.to_undirected()

In [ ]:
partition = community_louvain.best_partition(Gd)

In [ ]:
partition_df = pd.DataFrame(list(partition.items()), columns=['Node', 'Community'])


In [ ]:
partition_df.head()

,Node,Community
0,19099,0
1,461009,0
2,252634,11
3,250833,11
4,418937,0


In [ ]:
partition_df.to_csv('/content/drive/My Drive/farcaster_bigquery/louvain_partition.csv')
partition_df.to_parquet('/content/drive/My Drive/farcaster_bigquery/louvain_partition.parquet')

In [ ]:
partition_df= pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/louvain_partition.parquet')

In [ ]:
partition_df.size

820208

In [ ]:
partition_df.head()

,fid,Community
0,19099,0
1,461009,0
2,252634,11
3,250833,11
4,418937,0


In [ ]:
partition_df.size

820208

In [ ]:
partition_df.sort_values(by = ['fid'])

,fid,Community
2692,1,11
2696,2,4
1867,3,4
60964,4,5
40459,5,5
...,...,...
409012,542518,4
402238,542519,5
381333,542524,11
401999,542527,4


In [ ]:
partition_csv_df = pd.read_csv('/content/drive/My Drive/farcaster_bigquery/louvain_partition.csv')
partition_parquet_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/louvain_partition.parquet')

In [ ]:
partition_csv_df.size

1230312

In [ ]:
partition_parquet_df.size

820208

# Run Pagerank on the fids




In [ ]:
pr = nx.pagerank(G)

# Process results to find top 10 fids by pagerank

In [ ]:
df_pr = pd.DataFrame.from_dict(pr,  orient='index', columns = ['pagerank']).reset_index(names=['fid'])

In [ ]:
df_pr

,fid,pagerank
0,19099,1.225519e-05
1,461009,2.612976e-06
2,252634,6.516458e-06
3,250833,1.405970e-06
4,418937,1.592415e-06
...,...,...
410099,189247,4.503864e-07
410100,185724,4.503864e-07
410101,164852,4.503864e-07
410102,248489,4.503864e-07


In [ ]:
pr_sorted_df = df_pr.sort_values(by='pagerank', ascending=False)

In [ ]:
# need outer
pr_sorted_profiles_df = pd.merge(profiles_df, pr_sorted_df, on = 'fid').sort_values('pagerank', ascending=False)

In [ ]:
# pr_sorted_profiles_df.to_parquet('/content/drive/My Drive/farcaster_bigquery/pagerank_profiles.parquet')
# pr_sorted_profiles_df.to_csv('/content/drive/My Drive/farcaster_bigquery/pagerank_profiles.csv')
pr_sorted_profiles_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/pagerank_profiles.parquet')

# Map the pagerank to a 50-99 scale

In [ ]:
# Scaling from 50-99
min_target = 50
max_target = 99

original_min = pr_sorted_profiles_df['pagerank'].min()
original_max = pr_sorted_profiles_df['pagerank'].max()

pr_sorted_profiles_df['pr_scaled_50_99'] = round((pr_sorted_profiles_df['pagerank'] - original_min) * (max_target - min_target) / (original_max - original_min) + min_target, 0)

In [ ]:
# Scarling from 1-99
min_target = 1
max_target = 99

original_min = pr_sorted_profiles_df['pagerank'].min()
original_max = pr_sorted_profiles_df['pagerank'].max()

pr_sorted_profiles_df['pr_scaled_1_99'] = round((pr_sorted_profiles_df['pagerank'] - original_min) * (max_target - min_target) / (original_max - original_min) + min_target,0)

In [ ]:
pr_sorted_profiles_df['pr_scaled_1_99_lt50is50'] = pr_sorted_profiles_df['pr_scaled_1_99'].apply(lambda x: max(x, 50))

In [ ]:
pr_sorted_profiles_df.head()

,fid,last_updated_at,data,custody_address,date,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50
381284,3,2023-08-31 16:52:50+00:00,"{""bio"":""Working on Farcaster and Warpcast."",""d...",None,2023-08-31,0.006092,99.0,99.0,99.0
392789,5650,2024-01-19 01:20:58+00:00,"{""bio"":""hullo"",""display"":""Vitalik Buterin"",""pf...",None,2024-01-19,0.005517,90.0,94.0,90.0
263907,99,2024-04-16 01:55:58+00:00,"{""bio"":""@base contributor #001; onchain cities...",None,2024-04-16,0.004769,78.0,88.0,78.0
371557,2,2024-05-06 21:10:43+00:00,"{""bio"":""Technowatermelon. Elder Millenial. Bui...",None,2024-05-06,0.004576,75.0,87.0,75.0
260989,207,2024-04-12 20:29:26+00:00,"{""bio"":""(summer of) protocol {support | guild ...",None,2024-04-12,0.004384,72.0,85.0,72.0


In [ ]:
pr_sorted_profiles_df.data.fillna('{}', inplace=True)

In [ ]:
# Function to parse JSON strings
def parse_json_column(json_str):
    return pd.Series(json.loads(json_str))

# Apply the function to the 'Profile' column
expanded_columns = pr_sorted_profiles_df['data'].apply(parse_json_column)

# Concatenate the original DataFrame with the new columns
scores_df = pd.concat([pr_sorted_profiles_df, expanded_columns], axis=1)

# Drop the original 'Profile' column
scores_df.drop(columns=['data', 'url'], inplace=True)

In [ ]:
scores_df['percentile'] = round(scores_df['pagerank'].rank(pct=True) * 99)
scores_df['percentile_50_99'] = scores_df['percentile'].apply(lambda x: max(x, 50))

In [ ]:
scores_df['percentile_50_99']

381284    99.0
392789    99.0
263907    99.0
371557    99.0
260989    99.0
          ... 
179411    50.0
179394    50.0
327306    50.0
179310    50.0
249113    50.0
Name: percentile_50_99, Length: 407054, dtype: float64

In [ ]:
scores_df = pd.read_csv('/content/drive/My Drive/farcaster_bigquery/pagerank_percentile.csv')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0,1,2,3,4,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:

scores_df.size

10995660

In [ ]:
# scores_df[0:100].to_csv('/content/drive/My Drive/farcaster_bigquery/pagerank_top100.csv')

In [ ]:
scores_df.head()

,fid,last_updated_at,custody_address,date,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50,bio,display,pfp,username,percentile,percentile_50_99
381284,3,2023-08-31 16:52:50+00:00,None,2023-08-31,0.006092,99.0,99.0,99.0,Working on Farcaster and Warpcast.,Dan Romero,https://res.cloudinary.com/merkle-manufactory/...,dwr.eth,99.0,99.0
392789,5650,2024-01-19 01:20:58+00:00,None,2024-01-19,0.005517,90.0,94.0,90.0,hullo,Vitalik Buterin,https://i.imgur.com/IzJxuId.jpg,vitalik.eth,99.0,99.0
263907,99,2024-04-16 01:55:58+00:00,None,2024-04-16,0.004769,78.0,88.0,78.0,@base contributor #001; onchain cities w/ OAK ...,Jesse Pollak 🔵,https://i.imgur.com/rOy7TtZ.gif,jessepollak,99.0,99.0
371557,2,2024-05-06 21:10:43+00:00,None,2024-05-06,0.004576,75.0,87.0,75.0,Technowatermelon. Elder Millenial. Building Fa...,Varun Srinivasan,https://i.seadn.io/gae/sYAr036bd0bRpj7OX6B-F-M...,v,99.0,99.0
260989,207,2024-04-12 20:29:26+00:00,None,2024-04-12,0.004384,72.0,85.0,72.0,(summer of) protocol {support | guild | fellow...,timbeiko.eth,https://lh3.googleusercontent.com/G2z3yvOBkfHn...,tim,99.0,99.0


In [ ]:
scores_df.size

10995660

In [ ]:
partition_df = partition_df.rename(columns={'Node': 'fid'})

In [ ]:
scores_comms_df = pd.merge(scores_df, partition_df, on = 'fid', how='outer', left_index=False, right_index=False)

In [ ]:
scores_comms_verify_df = pd.merge(scores_comms_df, verified_df, on = 'fid')

In [ ]:
scores_comms_verify_df.size

3345372

In [ ]:
scores_df.size

10995660

In [ ]:
partition_df.size

820208

In [ ]:
scores_comms_verify_df.to_csv('/content/drive/My Drive/farcaster_bigquery/scores_comms_verify.csv')

In [ ]:
scores_df.describe()

,fid,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50,percentile,percentile_50_99
count,407054.0,4.070540e+05,407054.000000,407054.000000,407054.000000,407054.000000,407054.000000
mean,326442.712999,2.452689e-06,1.022093,50.009814,50.000531,49.492544,62.126325
std,134890.81742,3.813109e-05,0.613998,0.306662,0.124169,28.573282,15.784895
min,1.0,4.469975e-07,1.000000,50.000000,50.000000,0.000000,50.000000
25%,232044.25,4.529686e-07,1.000000,50.000000,50.000000,25.000000,50.000000
50%,335054.5,4.768670e-07,1.000000,50.000000,50.000000,50.000000,50.000000
75%,437403.75,8.084265e-07,1.000000,50.000000,50.000000,74.000000,74.000000
max,539339.0,6.091768e-03,99.000000,99.000000,99.000000,99.000000,99.000000


In [ ]:
scores_df[0:100]

,fid,last_updated_at,custody_address,date,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50,bio,display,pfp,username,percentile,percentile_50_99
381284,3,2023-08-31 16:52:50+00:00,None,2023-08-31,0.006092,99.0,99.0,99.0,Working on Farcaster and Warpcast.,Dan Romero,https://res.cloudinary.com/merkle-manufactory/...,dwr.eth,99.0,99.0
392789,5650,2024-01-19 01:20:58+00:00,None,2024-01-19,0.005517,90.0,94.0,90.0,hullo,Vitalik Buterin,https://i.imgur.com/IzJxuId.jpg,vitalik.eth,99.0,99.0
263907,99,2024-04-16 01:55:58+00:00,None,2024-04-16,0.004769,78.0,88.0,78.0,@base contributor #001; onchain cities w/ OAK ...,Jesse Pollak 🔵,https://i.imgur.com/rOy7TtZ.gif,jessepollak,99.0,99.0
371557,2,2024-05-06 21:10:43+00:00,None,2024-05-06,0.004576,75.0,87.0,75.0,Technowatermelon. Elder Millenial. Building Fa...,Varun Srinivasan,https://i.seadn.io/gae/sYAr036bd0bRpj7OX6B-F-M...,v,99.0,99.0
260989,207,2024-04-12 20:29:26+00:00,None,2024-04-12,0.004384,72.0,85.0,72.0,(summer of) protocol {support | guild | fellow...,timbeiko.eth,https://lh3.googleusercontent.com/G2z3yvOBkfHn...,tim,99.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381286,302,2023-12-22 04:45:24+00:00,None,2023-12-22,0.001154,20.0,59.0,50.0,Working on Farcaster.,Goksu Toprak,https://i.imgur.com/YM5spql.jpg,gt,99.0,99.0
372249,4407,2023-12-05 22:52:08+00:00,None,2023-12-05,0.001150,19.0,59.0,50.0,"The Farcaster formerly known as Kchamp. Books,...",Katherine,https://i.imgur.com/kynnpYw.jpg,keccers.eth,99.0,99.0
404851,9816,2024-05-09 10:46:16+00:00,None,2024-05-09,0.001122,19.0,59.0,50.0,🇫🇷 Generative artist • he/him • \nNFTs on gm. ...,Camille Roux,https://i.imgur.com/KIWqYJC.jpg,camilleroux,99.0,99.0
400878,1689,2024-02-13 22:13:17+00:00,None,2024-02-13,0.001118,19.0,59.0,50.0,"wannabe cypherpunk, engineer working on @frame...",Stephan,https://i.imgur.com/njEtQOM.jpg,stephancill,99.0,99.0


In [ ]:
scores_df.sample(n=100)

,fid,last_updated_at,custody_address,date,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50,bio,display,pfp,username,percentile,percentile_50_99
216491,186205,NaT,None,None,4.503864e-07,1.0,50.0,50.0,NaN,NaN,NaN,NaN,12.0,50.0
301205,200400,2023-12-08 20:51:33+00:00,None,2023-12-08,4.545731e-07,1.0,50.0,50.0,Fit via vi,JT,https://i.imgur.com/cNKn7bT.jpg,jtchristy,28.0,50.0
128326,512768,2024-05-09 07:48:38+00:00,0xb4556f353b5cba3eb388193c629a5a4700d5254e,2024-05-09,1.249790e-06,1.0,50.0,50.0,Ma name Sheyda,Shyda,https://imagedelivery.net/BXluQx4ige9GuW0Ia56B...,NaN,81.0,81.0
165444,389985,2024-04-16 17:59:37+00:00,0xb9f3a09ea02c699298db900ed544b5b8182fd732,2024-04-16,9.428426e-07,1.0,50.0,50.0,crypto✨,Rey,https://imagedelivery.net/BXluQx4ige9GuW0Ia56B...,winter3,77.0,77.0
27503,414802,2024-03-28 17:39:22+00:00,0x2985b820f13373cf5baf5b400af2fb6ae4d07ff6,2024-03-28,4.594605e-07,1.0,50.0,50.0,,Octopus,https://i.imgur.com/IUlfS5w.jpg,octopusonbush,36.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213007,181689,NaT,None,None,4.503864e-07,1.0,50.0,50.0,NaN,NaN,NaN,NaN,12.0,50.0
202373,176729,NaT,None,None,4.503864e-07,1.0,50.0,50.0,NaN,NaN,NaN,NaN,12.0,50.0
119479,435185,2024-04-01 13:53:03+00:00,0xde840ce806499a84effc6ea91a28c24ffbac5ca3,2024-04-01,5.359724e-07,1.0,50.0,50.0,"I’m pepe artist, check me on X \n@BillyNFTees",BillyNFTees,https://i.imgur.com/yVrNhrS.jpg,christianbicasso,62.0,62.0
391048,274187,2024-02-11 01:26:20+00:00,None,2024-02-11,1.051224e-06,1.0,50.0,50.0,"have a joint family in which my uncle, grandpa...",Manish Sutariya,https://dego.finance/upload/big/1.png,manishr,79.0,79.0
